In [1]:
%pip install -q --no-warn-conflicts malariagen_data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.6/160.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 24.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.5/302.5 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.4/24.4 MB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.2/210.2 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import malariagen_data

In [3]:
ag3 = malariagen_data.Ag3()
ag3

<MalariaGEN Ag3 API client>
Storage URL             : gs://vo_agam_release_master_us_central1
Data releases available : 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8
Results cache           : None
Cohorts analysis        : 20231215
AIM analysis            : 20220528
Site filters analysis   : dt_20200416
Software version        : malariagen_data 10.0.0
Client location         : Iowa, United States (Google Cloud us-central1)
---
Please note that data are subject to terms of use,
for more information see https://www.malariagen.net/data
or contact support@malariagen.net. For API documentation see 
https://malariagen.github.io/malariagen-data-python/v10.0.0/Ag3.html

In [4]:
ds_haps = ag3.haplotypes(
    region="2L:3,745,921-3,747,552",
    analysis="gamb_colu_arab",
    sample_sets = '3.0',
)
ds_haps

<xarray.Dataset>
Dimensions:           (variants: 481, alleles: 2, samples: 2784, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(481,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(481,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(481, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(481, 64, 2), meta=np.ndarray>
Attributes:
    contigs:   ('2R', '2L', '3R', '3L', 'X')
    analysis:  gamb_colu_arab

In [5]:
n_samples = ds_haps.sizes["samples"]
n_samples

2784

In [6]:
gt = ds_haps["call_genotype"].values
gt.shape

(481, 2784, 2)

In [58]:
import allel

In [59]:
# access to genotypes array for all data

genotyp = allel.GenotypeDaskArray(ds_haps['call_genotype'].data)
genotyp

<GenotypeDaskArray shape=(481, 2784, 2) dtype=int8>

In [10]:
# Convert GenotypeDaskArray to HaplotypeArray

haplotyp = allel.HaplotypeArray(genotyp.to_haplotypes())
haplotyp

<HaplotypeArray shape=(481, 5568) dtype=int8>
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
...
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0
0 0 0 0 0 ... 0 0 0 0 0

In [60]:
# Haplotype diversity

allel.haplotype_diversity(haplotyp)

0.9751673836379816

In [61]:
ds_haps.sizes["samples"]

2784

In [95]:
# Let's try with Angola sample
ds_haps_coluzzii_AN = ag3.haplotypes(
    region="2L:3,745,921-3,747,552",
    analysis="gamb_colu_arab",
    sample_sets='3.0',
    sample_query="taxon == 'coluzzii' and country == 'Angola'"
)
ds_haps_coluzzii_AN

<xarray.Dataset>
Dimensions:           (variants: 481, alleles: 2, samples: 81, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(481,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(481,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(81,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(481, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(481, 64, 2), meta=np.ndarray>
Attributes:
    contigs:   ('2R', '2L', '3R', '3L', 'X')
    analysis:  gamb_colu_arab

In [96]:
genotyp_AN = allel.GenotypeDaskArray(ds_haps_coluzzii_AN['call_genotype'].data)
haplotyp_AN = allel.HaplotypeArray(genotyp_AN.to_haplotypes())
allel.haplotype_diversity(haplotyp_AN)

0.7747872095698183

In [100]:
# Let's try with Burkina Faso sample (gambiae)
ds_haps_gambiae_BFA = ag3.haplotypes(
    region="2L:3,745,921-3,747,552",
    analysis="gamb_colu_arab",
    sample_sets='3.0',
    sample_query="taxon == 'gambiae' and country == 'Burkina Faso'"
)
ds_haps_gambiae_BFA

<xarray.Dataset>
Dimensions:           (variants: 481, alleles: 2, samples: 158, ploidy: 2)
Coordinates:
    variant_position  (variants) int32 dask.array<chunksize=(481,), meta=np.ndarray>
    variant_contig    (variants) uint8 dask.array<chunksize=(481,), meta=np.ndarray>
    sample_id         (samples) object dask.array<chunksize=(99,), meta=np.ndarray>
Dimensions without coordinates: variants, alleles, samples, ploidy
Data variables:
    variant_allele    (variants, alleles) |S1 dask.array<chunksize=(481, 1), meta=np.ndarray>
    call_genotype     (variants, samples, ploidy) int8 dask.array<chunksize=(481, 34, 2), meta=np.ndarray>
Attributes:
    contigs:   ('2R', '2L', '3R', '3L', 'X')
    analysis:  gamb_colu_arab

In [98]:
genotyp_BFA = allel.GenotypeDaskArray(ds_haps_gambiae_BFA['call_genotype'].data)
haplotyp_BFA = allel.HaplotypeArray(genotyp_BFA.to_haplotypes())
allel.haplotype_diversity(haplotyp_BFA)

0.852401044806108